In [99]:
import pandas as pd
import csv
from functools import reduce


def is_norm(tr):
    """ Checks if traceroute was parsed normally. """
    return len(tr) == 19

def makeDict(x):
    """ Create a dictionary from a list"""
    if len(x) > 2: raise Exception('Was only expecting 2 elements')
    i = iter(x)
    b = dict(zip(i, i))
    
    return b


with open('FT.TXT') as f:
    reader = csv.reader(f, delimiter='\n')
    dat = []
    a = []
    for row in reader:
        if row != []:
            a.extend(row)
        elif row == []:
            dat.append(a)
            a = []

r1 = dat[0]
dat[0]

['start: 15:33:47',
 'traceroute to google.com (216.58.194.142), 64 hops max',
 '  1   192.168.1.254  1.316ms  1.881ms  1.068ms ',
 '  2   75.12.92.1  22.182ms  21.474ms  18.137ms ',
 '  3   *  71.152.134.45  1619.579ms  1898.068ms ',
 '  4   12.123.7.142  29.078ms  0.005ms  39.652ms ',
 '  5   12.122.133.45  27.729ms  1.219ms  27.187ms ',
 '  6   12.255.10.44  29.281ms  0.063ms  27.117ms ',
 '  7   *  *  * ',
 '  8   108.170.238.138  27.849ms  0.333ms  26.864ms ',
 '  9   108.170.243.165  28.434ms  108.170.244.2  0.003ms  108.170.243.165  28.225ms ',
 ' 10   209.85.241.123  29.356ms  209.85.254.94  2.599ms  209.85.241.123  25.552ms ',
 ' 11   209.85.249.19  51.129ms  0.380ms  51.256ms ',
 ' 12   209.85.250.46  50.403ms  0.436ms  50.200ms ',
 ' 13   216.239.63.206  48.156ms  108.170.228.90  1.302ms  216.239.63.206  45.911ms ',
 ' 14   108.170.252.161  49.301ms  108.170.252.129  0.182ms  108.170.252.161  49.686ms ',
 ' 15   108.170.230.117  50.612ms  1.102ms  49.235ms ',
 ' 16   216.58.

In [100]:
if is_norm(r1):
    START, END = r1[0], r1[18]
    WHERE_TO = r1[1]
    CHUNK = r1[2:18]
    print('Start: {}, \nEnd: {}, \nDestination: {}'.format(START, END, WHERE_TO))
CHUNK

Start: start: 15:33:47, 
End: end: 15:34:04, 
Destination: traceroute to google.com (216.58.194.142), 64 hops max


['  1   192.168.1.254  1.316ms  1.881ms  1.068ms ',
 '  2   75.12.92.1  22.182ms  21.474ms  18.137ms ',
 '  3   *  71.152.134.45  1619.579ms  1898.068ms ',
 '  4   12.123.7.142  29.078ms  0.005ms  39.652ms ',
 '  5   12.122.133.45  27.729ms  1.219ms  27.187ms ',
 '  6   12.255.10.44  29.281ms  0.063ms  27.117ms ',
 '  7   *  *  * ',
 '  8   108.170.238.138  27.849ms  0.333ms  26.864ms ',
 '  9   108.170.243.165  28.434ms  108.170.244.2  0.003ms  108.170.243.165  28.225ms ',
 ' 10   209.85.241.123  29.356ms  209.85.254.94  2.599ms  209.85.241.123  25.552ms ',
 ' 11   209.85.249.19  51.129ms  0.380ms  51.256ms ',
 ' 12   209.85.250.46  50.403ms  0.436ms  50.200ms ',
 ' 13   216.239.63.206  48.156ms  108.170.228.90  1.302ms  216.239.63.206  45.911ms ',
 ' 14   108.170.252.161  49.301ms  108.170.252.129  0.182ms  108.170.252.161  49.686ms ',
 ' 15   108.170.230.117  50.612ms  1.102ms  49.235ms ',
 ' 16   216.58.194.142  47.870ms  1.357ms  46.211ms ']

In [101]:
split_up = [x.split('   ') for x in CHUNK]
result_dict = reduce(lambda a, b: {**a, **b}, [makeDict(x) for x in split_up])
result_dict

{'  1': '192.168.1.254  1.316ms  1.881ms  1.068ms ',
 '  2': '75.12.92.1  22.182ms  21.474ms  18.137ms ',
 '  3': '*  71.152.134.45  1619.579ms  1898.068ms ',
 '  4': '12.123.7.142  29.078ms  0.005ms  39.652ms ',
 '  5': '12.122.133.45  27.729ms  1.219ms  27.187ms ',
 '  6': '12.255.10.44  29.281ms  0.063ms  27.117ms ',
 '  7': '*  *  * ',
 '  8': '108.170.238.138  27.849ms  0.333ms  26.864ms ',
 '  9': '108.170.243.165  28.434ms  108.170.244.2  0.003ms  108.170.243.165  28.225ms ',
 ' 10': '209.85.241.123  29.356ms  209.85.254.94  2.599ms  209.85.241.123  25.552ms ',
 ' 11': '209.85.249.19  51.129ms  0.380ms  51.256ms ',
 ' 12': '209.85.250.46  50.403ms  0.436ms  50.200ms ',
 ' 13': '216.239.63.206  48.156ms  108.170.228.90  1.302ms  216.239.63.206  45.911ms ',
 ' 14': '108.170.252.161  49.301ms  108.170.252.129  0.182ms  108.170.252.161  49.686ms ',
 ' 15': '108.170.230.117  50.612ms  1.102ms  49.235ms ',
 ' 16': '216.58.194.142  47.870ms  1.357ms  46.211ms '}